# S02T02: Geração dos espectrogramas para todo o dataset 

## The one with the libs

In [6]:
import numpy as np
import librosa
from scipy import signal 
import glob
import re 
import pickle
from tqdm import tqdm
import os
path = "/home/arlaxad/Documentos/Sistemas de Informação/2020.3/base_de_dados/extendedballroom_v1 (cópia).1(.wav)/Treino/"
print(path) 

/home/arlaxad/Documentos/Sistemas de Informação/2020.3/base_de_dados/extendedballroom_v1 (cópia).1(.wav)/Treino/


## The one with mel-spectrogram 201 x 201

In [2]:
def compute_mel(audioname):
    wav = librosa.load(audioname, sr=22050, duration=3.5)[0]
    melspec = librosa.feature.melspectrogram(wav,sr=22050,n_fft=1024,hop_length=385,n_mels=201)
    logmel = librosa.core.power_to_db(melspec)
    
    # Créditos: Jederson
    max_pad_len = 201
    pad_width = max_pad_len - logmel.shape[1]
    logmel = np.pad(logmel, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    return logmel

In [ ]:
stft_datas = []

In [ ]:
def compute_stft(audioname):
         
        wav = librosa.load(audioname, sr=22050, duration=5.0)[0]
        stftspec = np.abs(librosa.stft(wav, n_fft=500,  hop_length=440, window=signal.blackmanharris(500)))
         
        logstft = librosa.core.power_to_db(stftspec**2)
        
        stft_datas.append(stftspec)
        
        return logstft

In [ ]:
def compute_harmonic(id_):
    kernel_size=31
    power=2.0
    mask=False
    margin=1.0
    
    harmspec, P = librosa.decompose.hpss(stft_datas[id_], kernel_size=kernel_size, power=power, mask=mask, margin=margin)

    logharm = librosa.core.power_to_db(harmspec)

    return logharm

## The one with the audio name and class 

In [3]:
def get_audio_name_class(path):
    audio_name = path.split('/')[-1]
#     print (audio_name)
    audio_name_wo_ext = re.sub(r'\.wav', "", audio_name)
#     print (audio_name_wo_ext)
    label = path.split('/')[-2]
#     print (label)
    return audio_name_wo_ext, label

## The one who saves the specs

In [7]:
def run(dataset_path, pickle_name):
    
    #Captura os nomes das pastas
    name_folders = glob.glob(dataset_path + '*')
    
    mel_x = []
    mel_y = []
#     stft_x = []
#     stft_y = []
#     harm_x = []
#     harm_y = []
    
    for names_ in name_folders:
        #Captura o caminho completo do audio
        path_audios = glob.glob(names_ + '/*.wav')
        
        for id_,path_ in enumerate(tqdm(path_audios)):
            audio_name, label = get_audio_name_class(path_)
            logmel = compute_mel(path_)
#             logstft = compute_stft(path_)
#             logharm = compute_harmonic(id_)
            
            mel_x.append(logmel)
            mel_y.append(label)
            
#             stft_x.append(logstft)
#             stft_y.append(label)
            
#             harm_x.append(logharm)
#             harm_y.append(label)
                
    pickle.dump(mel_x, open(dataset_path + "/"+pickle_name+'_x_' + str(pickle_name) +'_mel.pickle', 'wb'))
    pickle.dump(mel_y, open(dataset_path +"/"+pickle_name+'_y_' + str(pickle_name) +'_mel.pickle', 'wb'))
    
#     pickle.dump(stft_x, open(dataset_path + "/"+pickle_name+'_x_' + str(pickle_name) +'_stft.pickle', 'wb'))
#     pickle.dump(stft_y, open(dataset_path +"/"+pickle_name+'_y_' + str(pickle_name) +'_stft.pickle', 'wb'))
    
#     pickle.dump(harm_x, open(dataset_path + "/"+pickle_name+'_x_' + str(pickle_name) +'_harm.pickle', 'wb'))
#     pickle.dump(harm_y, open(dataset_path +"/"+pickle_name+'_y_' + str(pickle_name) +'_harm.pickle', 'wb'))

In [8]:
run(path, 'Treino') 

100%|██████████| 1715/1715 [08:53<00:00,  3.21it/s]


## The one who checks the pickles 

In [ ]:
def read_pickle(name):
    with (open(name, 'rb')) as openfile:
        while True:
            try:
                one_instance = pickle.load(openfile)
            except EOFError:
                break
    one_instance = np.asanyarray(one_instance)
    return one_instance

In [ ]:
x_test_harm = read_pickle(path+'/Treino_x_Treino_harm.pickle')
y_test_harm = read_pickle(path+'/Treino_y_Treino_harm.pickle')
# x_test_stft = read_pickle(path+'/Treino_x_Treino_stft.pickle')
# y_test_stft = read_pickle(path+'/Treino_y_Treino_stft.pickle')
# x_test_mel = read_pickle(path+'/Treino_x_Treino_mel.pickle')
# y_test_mel = read_pickle(path+'/Treino_y_Treino_mel.pickle')


In [ ]:
print(x_test_stft.shape)
# print(x_test_harm.shape)
# print(x_test_mel.shape)